In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#### 1. Dataset
Let's prepare our dataset.

In [2]:
# download Shakespeare
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-12 22:13:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2024-07-12 22:13:13 (44.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [16]:
torch.manual_seed(1337)

class DataSet():
    def __init__(self):
        with open('input.txt', 'r', encoding='utf-8') as f:
            text = f.read()

        chars = sorted(list(set(text)))
        self.vocab_size = len(chars)

        # 1. encode: take a string, output a list of integers
        ctoi = {ch:i for i, ch in enumerate(chars)}
        encode = lambda s : [ ctoi[ch] for ch in s]

        # 2. decode: take a list of integers, output a string
        itoc = {i:ch for i, ch in enumerate(chars)}
        self.decode = lambda l : ''.join([ itoc[i] for i in l])

        # 3. encode the entire text and store it in a torch.Tensor
        self.data = encode(text)
        self.data = torch.tensor(self.data, dtype=torch.long)

    def __getitem__(self, idx):
        """given an idx, return a sample of the DataSet at this given idx"""
        return self.data[idx]

    def train_dataset(self):
        n = int(0.9*len(self.data))
        return self.data[:n]

    def val_dataset(self):
        n = int(0.9*len(self.data))
        return self.data[n:]

#### 2. DataLoader
DataLoader randomly samples mini-batches from DataSet.

In [4]:
torch.manual_seed(1337)

class DataLoader:
    def __init__(self, dataset, batch_size=4, block_size=8):
        self.dataset = dataset
        self.batch_size = batch_size
        self.block_size = block_size

    def get_batch(self, split):

        data = self.dataset.train_dataset() if split == 'train' else self.dataset.val_dataset()

        # generate a tensor with random numbers
        idx = torch.randint(len(data) - self.block_size, (self.batch_size,))

        x = torch.stack([data[i:i+self.block_size] for i in idx])
        y = torch.stack([data[i+1:i+self.block_size+1] for i in idx])
        x, y = x.to(device), y.to(device)
        return x, y

#### 3. GPT
let's build a GPT from scratch.

In [5]:
# in the code comment,
# B, stands for batch_size
# T, stands for block_size
# C, stands for embedding size (i.e. n_embd)

class Head(nn.Module):
    """one head of self-attention"""
    def __init__(self, n_embd, head_size, block_size, dropout):
        super().__init__()

        # note 1: weight initialization is important. it happens inside nn.Linear(...) module
        # note 2: n_embd is the size of each embedding vector
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        # regularization
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x is of (B, T, C)

        B,T,C = x.shape

        # k is what this token contains
        k = self.key(x)   # (B,T,C)

        # q is what this token is looking for
        q = self.query(x) # (B,T,C)

        # compute attention score.
        # attention score is "affinity"
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, T)

        # mask, so that a token can't see the future (i.e. a token can attend to the tokens before it)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)

        wei = F.softmax(wei, dim=-1) # (B, T, T)

        # drop out
        wei = self.dropout(wei)

        v = self.value(x) # (B, T, C)

        # perform the weighted aggregation of the values
        out = wei @ v # (B, T, C)

        return out

In [6]:
class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, n_embd, head_size, block_size, dropout):
        super().__init__()

        self.heads = nn.ModuleList([ Head(n_embd, head_size, block_size, dropout) for _ in range(num_heads)])

        # pre-condition: head_size * num_heads == n_embd
        self.proj = nn.Linear(n_embd, n_embd) # the projection layer going back to the residual pathway

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [7]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # the projection layer going back to the residual pathway
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [8]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, num_heads, n_embd, block_size, dropout):
        super().__init__()
        head_size = n_embd // num_heads
        self.sa = MultiHeadAttention(num_heads, n_embd, head_size, block_size, dropout) # communication

        # LayerNorm (critial for optimization)
        self.ln1 = nn.LayerNorm(n_embd)

        self.ffwd = FeedForward(n_embd, dropout) # computation

        # LayerNorm (critial for optimization)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # Residual Connection applies here
        x = x + self.ffwd(self.ln2(x))  # Residual Connection applies here
        return x

In [9]:
class GPT(nn.Module):

    def __init__(self, vocab_size, num_heads, n_embd, block_size, dropout, n_layer):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

        # note: the block_size needs to be defined, for position embedding table
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        self.blocks = nn.Sequential(*[Block(num_heads, n_embd, block_size, dropout) for _ in range(n_layer)])

        self.ln_f = nn.LayerNorm(n_embd) # final layer norm

        self.lm_head = nn.Linear(n_embd, vocab_size)


    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,C)

        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)

        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

#### 4. Train GPT
let's train it.

In [10]:
batch_size = 16
block_size = 32
num_heads = 4
n_embd = 64
block_size = 32
dropout = 0.0
n_layer = 4
learning_rate = 1e-3
epochs = 5000
eval_interval = 100
eval_iters = 200

# 1. create a DataLoader
dataset = DataSet()
vocab_size = dataset.vocab_size
dataloader = DataLoader(dataset, batch_size, block_size)

# 2. create a GPT
gpt = GPT(vocab_size, num_heads, n_embd, block_size, dropout, n_layer)
gpt = gpt.to(device) # move the model's parameters to device

# print the number of parameters in the model
print(sum(p.numel() for p in gpt.parameters())/1e6, 'M parameters')

# 3. create a Optimizer
optimizer = torch.optim.AdamW(gpt.parameters(), lr=learning_rate)

# 4. create an epoch function, which takes dataloader, gpt model, and optimizer, then run 1 epoch
def epoch(dataloader, gpt, optimizer):
    # sample a batch of data
    xb, yb = dataloader.get_batch('train')

    # forward
    logits, loss = gpt(xb, yb)

    # clear out the previous gradient, if there is any
    optimizer.zero_grad(set_to_none=True)

    # backward
    loss.backward()

    # update paramaters
    optimizer.step()

# 5. train

# tell torch's manager that everything inside this code block, we will never call backward()
# so that torch will not store all the intermediate variables, so it is a lot of memory efficient
@torch.no_grad()
def estimate_loss():
    # average up the loss average multiple batches (i.e. eval_iters)
    out = {}
    gpt.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = dataloader.get_batch(split)
            logits, loss = gpt(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    gpt.train()
    return out

for iter in range(epochs):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == epochs - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    epoch(dataloader, gpt, optimizer)


0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5091, val loss 2.5058
step 300: train loss 2.4197, val loss 2.4336
step 400: train loss 2.3501, val loss 2.3562
step 500: train loss 2.2963, val loss 2.3125
step 600: train loss 2.2407, val loss 2.2496
step 700: train loss 2.2054, val loss 2.2187
step 800: train loss 2.1633, val loss 2.1866
step 900: train loss 2.1241, val loss 2.1504
step 1000: train loss 2.1036, val loss 2.1306
step 1100: train loss 2.0698, val loss 2.1180
step 1200: train loss 2.0380, val loss 2.0791
step 1300: train loss 2.0248, val loss 2.0634
step 1400: train loss 1.9926, val loss 2.0359
step 1500: train loss 1.9697, val loss 2.0287
step 1600: train loss 1.9627, val loss 2.0477
step 1700: train loss 1.9403, val loss 2.0115
step 1800: train loss 1.9090, val loss 1.9941
step 1900: train loss 1.9092, val loss 1.9858
step 2000: train loss 1.8847, val loss 1.9925
step 2100: train loss 1.

AttributeError: 'DataSet' object has no attribute 'decode'

In [17]:
# 6. generate
dataset = DataSet()
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(dataset.decode(gpt.generate(context, max_new_tokens=2000)[0].tolist()))



KING RICHARD II:
Shal lifest made to bub, to take Our my dagatants:
Whith foul his vetward that a endrer, my fears' to zorm heavens,
Oof it heart my would but
With ensengmin latest in ov the doest not.

WARWICK:
Welll now, and thus quechiry: there's speak you love.
In Bodiet, and whom the sclittle
Enout-now what evily well most rive with is compon to the me
Town danters, If so;
Ange to shall do aleous, for dear?

KING HENRY VI:
Hark, but a
ards bring Edward?

GROKE:
As is no Rurnts I am you! who neet.
Pom mary thou contrantym so a thense.

QUEEN VINCENTIO:
O, sir, may in God't well ow, whom confessy.
Which migh.

ARCHILINIUS:
Dithul seaze Peed me: very it passce of's cruport;
How what make you fear tals: there loves
Tunkistren in deed, is xment.

CORIONIUS:
What comforts me. I with self From the walt I?

GRINION:
Which ushold.

KING HENRY Gindner:
Withrief I doot, is onter now.

Securming:
Intande whose no crown some Eiverely marry sold;
For for me watch the
our torguet! Goy, know ou